In [ ]:
"""
    classification par RNN
"""

import numpy as np
import pandas as pd

from os import listdir
from os.path import isfile, join

from joblib import dump, load

In [ ]:
from google.colab import drive
drive.mount('/drive') # 4/1AA...7rs

Mounted at /drive


In [ ]:
def loadDataframes(machines, sets):

  dataFolder = '/drive/MyDrive/asdpy/data/original/dc2020task2'
  dataframes = []

  for machine in machines:
      for s in sets:
          saveName = dataFolder+'/'+machine+'/df_'+machine+'_'+s+'.joblib'
          df_partial = load(saveName)
          dataframes.append(df_partial)
          # print(df_partial.head())

  # concatenate all partials dataframes on horizontal axis
  return pd.concat(dataframes, axis=0)

In [ ]:
# load datas
machines = ['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']
sets = ['train', 'test']

# concatenate all dataframes as one
df_train = loadDataframes(machines, ['train', 'test'])

#df_train['anomaly'] = df_train['anomaly'] + 1

#anomaly_dummies = pd.get_dummies(df_train['anomaly'], 'anomaly')
#df_train = df_train.join(anomaly_dummies)
df_train['anomaly'] = df_train['anomaly'].astype(str)
#df_train['anomaly_0'] = (df_train['anomaly_0']).to_string()
#df_train['anomaly_1'] = (df_train['anomaly_1']).to_string()

print(df_train.head())
print(df_train.shape)
print(df_train.dtypes)


                                         spectrogram anomaly
0  /drive/MyDrive/asdpy/data/original/dc2020task2...       0
1  /drive/MyDrive/asdpy/data/original/dc2020task2...       0
2  /drive/MyDrive/asdpy/data/original/dc2020task2...       0
3  /drive/MyDrive/asdpy/data/original/dc2020task2...       0
4  /drive/MyDrive/asdpy/data/original/dc2020task2...       0
(30987, 2)
spectrogram    object
anomaly        object
dtype: object


In [ ]:
# EXAM DEEP LEARNING

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    #shear_range = 0.2,
    #zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# On prend des lots de 32

#training_set = train_datagen.flow_from_directory('images',
#    target_size = (64, 64),
#    batch_size = 32,
#    class_mode = 'categorical'
#)

# doc: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
training_set = train_datagen.flow_from_dataframe(
  dataframe=df_train,
  directory=None,      # if None, data in x_col column should be absolute paths
  x_col="spectrogram", # column in dataframe that contains the filenames
  y_col="anomaly",     # column in dataframe that has the target data
  #subset="training",   # subset of data
  batch_size=32,       # optional random seed for shuffling and transformations
  color_mode='grayscale',
  seed=42,
  shuffle=True,
  class_mode="categorical", #"binary",
  target_size=(64,64)
)

# If class_mode="sparse", y_col="anomaly" column values must be strings
# If class_mode="categorical", y_col="['anomaly_0', 'anomaly_1']" column values must be type string, list or tuple
print("training_set : ")
print(training_set) # keras.preprocessing.image.DirectoryIterator object

print("")

test_set = train_datagen.flow_from_dataframe(
  dataframe=df_train,
  directory=None,      # if None, data in x_col column should be absolute paths
  x_col="spectrogram", # column in dataframe that contains the filenames
  y_col="anomaly",     # column in dataframe that has the target data
  #subset="validation", # subset of data
  batch_size=32,       # optional random seed for shuffling and transformations
  color_mode='grayscale',
  seed=42,
  shuffle=True,
  class_mode="categorical",
  target_size=(64,64)
)

print("test_set : ")
print(test_set) # keras.preprocessing.image.DirectoryIterator object

Found 30987 validated image filenames belonging to 2 classes.
training_set : 

Found 30987 validated image filenames belonging to 2 classes.
test_set : 


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout 
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D

# Instancie un nouveau modèle séquentiel appelé model à l'aide du constructeur Sequential
classifier = Sequential()

# Instancie une couche de convolution
layer_1 = Conv2D(
    filters=32,         # La couche sera composée de 32 matrices de convolution
    kernel_size=(3,3), # noyau de convolution de dimension 3*3
    padding='valid',    # pour que le noyau ne puisse pas dépasser les bords de l'image
    input_shape=(64, 64, 1), # image passée en entrée : 512px hauteur, 512px largeur et 1 canal
    activation='relu'
)

# Instancie une couche de max-pooling
layer_2 = MaxPooling2D(
    pool_size = (2, 2) # maximum calculé sur des morceaux de dimensions 2x2
)

# Instancie une couche de convolution
layer_3 = Conv2D(
    filters=32, kernel_size=(3, 3),
    padding='valid', activation='relu'
)

# Instancie une couche de max-pooling
layer_4 = MaxPooling2D(
    pool_size = (2, 2) # maximum calculé sur des morceaux de dimensions 2x2
)

# Instancie une couche d'applatissement
# Cette couche ne prend pas de paramètre et permet juste d'aplatir un vecteur en une matrice.
# Cette transformation est nécessaire pour la suite car les couches denses ne prennent que des vecteurs en entrée et non des matrices.
layer_5 = Flatten()

# Instancie une couche dense
layer_6 = Dense(
    units=128,          # nombre de neurones de sortie 
    activation='relu'   # fonction d'activation
)

# Instancie une couche dense
output_layer = Dense(
    units=2,             # nombre de neurones de sortie (2 classes)
    activation='softmax'  # fonction d'activation
)

# Ajout les couches aux modèle
classifier.add(layer_1)
classifier.add(layer_2)
classifier.add(layer_3)
classifier.add(layer_4)
classifier.add(layer_5)
classifier.add(layer_6)
classifier.add(output_layer)

classifier.summary()

classifier.compile(loss='categorical_crossentropy',  # fonction de perte
              optimizer='adam',                 # algorithme d'optimisation
              metrics=['accuracy'])             # métrique d'évaluation



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [ ]:
# https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/

batch_size = 32

history = classifier.fit_generator(epochs = 10,
                                   generator=training_set,
                                   validation_data=test_set,
                                   steps_per_epoch=50
)

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

print("train_acc : ", train_acc)
print("val_acc : ", val_acc)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 0.5174 - accuracy: 0.8239